 # IP Subnetting Calculator (Subnetting Operations)

In [13]:
#Code is explained with comments
import ipaddress #A built-in python module, which can read and calculate IP networks, addresses and masks.
import math

def get_network_input(choice): #asks the user for an INPUT in CIDR(Classless Inter-Domain Routing) format
    while True: #While condition will keep asking until the input is valid
        try: #"try", is a block that "tries" to run code that might fail, ensures no crash
            ip_cidr = input("Enter network in CIDR format (e.g., 192.168.1.0/24): ")
            network = ipaddress.IPv4Network(ip_cidr, strict=False) # tries to convert given input into a valid network object
            #"strict=False", if the user enters something like 192.168.1.7/24 then python will automatically correct it to the proper network base
            break # if input is valid, break runs and loop ends 
        # if input is invalid, except: runs and loop continues (asks for the valid input)
        except ValueError:#this catches the error instead of letting the program crash 
            print("Invalid CIDR format. Please try again.")
    
# Option 1: Ask for host per subnet
    if choice == '1':
        while True:#similar to the previous, instead asks for the number of hosts
            try:
                hosts_per_subnet = int(input("Enter required number of hosts per subnet: "))
                #if someone enters texts that are not numbers like "hi" or less than 1 numbers, we through an error with the code below
                if hosts_per_subnet < 1:
                    raise ValueError
                    # manually throws an error if the number is invalid, forcing the loop to continue
                break # if the input is valid, the loop ends 
            except ValueError : #"except" catches the error instead of letting the program crash, and displays the message below
                print("Invalid number of hosts. Please enter a positive integer.")
    
        #After both inputs are valid, they get returned as a tuple (immutable)
        return network, hosts_per_subnet
        
# Option 2: Ask for number of networks
# When the choice is 2, we have a similar format to choice 1
    else:
        while True:
            try:
                required_networks = int(input("Enter required number of networks: "))
                if required_networks < 1:
                    raise ValueError
                break
            except ValueError:
                print("Invalid number. Enter a positive integer.")
    
        return network, required_networks
        

#This function calculates how to split a network into smaller subnets that can each hold at least the given hosts (input)
def calculate_subnets(network, hosts_per_subnet): 
    #Calculate number of bits needed for hosts (+2 for network & broadcast)
    required_host_bits = math.ceil(math.log2(hosts_per_subnet + 2))
    #The reason we add 2 is to include network and broadcast addresses
    # math.log2()-tells us the number of bits needed to represent these addresses
    # math.ceil()- rounds up to nearest integer, in our case to the nearest bit
    new_prefix = 32 - required_host_bits #IPv4 = 32bits long
    #new_prefix is our new subnet mask, and the number of bits for our network
    
    if new_prefix < network.prefixlen:
        #checks whether subnetting is possible
        #network.prefixlen is the orginal networks prefix
        #the reason we put this condition is because we cannot create subnets with more hosts that the original network
        print(f"Cannot create subnets for {hosts_per_subnet} hosts in {network}.")
        return [], 0 #Thus we return 0, and an emty list

    #To generate the subnets with the new prefix, we use the function "network.subnets()"
    subnets = list(network.subnets(new_prefix=new_prefix))
    return subnets, new_prefix #Returns a list of subnets and the new prefix

#Borrows bits to satisfy number of subnets
def calculate_networks(network, required_networks):
    # Find smallest number of bits to borrow
    borrowed_bits = math.ceil(math.log2(required_networks))
    new_prefix = network.prefixlen + borrowed_bits
    
    if new_prefix > 30:  # /31 and /32 have no usable hosts
        print(f"Cannot create {required_networks} networks from {network}.")
        return [], 0
    
    subnets = list(network.subnets(new_prefix=new_prefix))
    return subnets[:required_networks], new_prefix


#This function prints a table of subnet details 
def display_subnets(subnets):
    if not subnets: # if the subnets is empty([]), the function returns nothing
        return
    #print a table header, <=: left align, number: width of column
    print("\n{:<10} {:<15} {:<15} {:<15} {:<25} {:<15}".format(
        "Subnet", "Network", "Broadcast", "Subnet Mask", "Valid Hosts", "Usable Hosts"
    ))
    print("-"*95) #the line '-' separates header from data

    #loop through each subnet
    for i, subnet in enumerate(subnets): #mention one by one, gives both index and the subnet
        #To get valid host range, we first convert it into a list to access the first and last host
        hosts = list(subnet.hosts())
        if hosts:
            host_range = f"{hosts[0]} - {hosts[-1]}"
        else:
            host_range = "None"
        
        total_hosts = subnet.num_addresses #total IPs in the subnet, including network and broadcast
        usable_hosts = total_hosts - 2 if total_hosts > 2 else total_hosts #subtracted network and broadcast, and if its less than 2 we just print total_hosts 
        
        #Print Subnet details
        # Subnet number, netwrod address, broadcast address, subnet mask, first to last usable IP, number of usable hosts
        print("{:<6} {:<15} {:<15} {:<15} {:<25} {:<15}".format(
            i+1, str(subnet.network_address), str(subnet.broadcast_address),
            str(subnet.netmask), host_range, usable_hosts
        ))

#Same logic as display of subnets
def display_networks(subnets):
    if not subnets:
        return
    
    print("\n{:<6} {:<15} {:<15} {:<15}".format(
        "Net#", "Network", "Broadcast", "Subnet Mask"
    ))
    print("-"*90)
    
    for i, subnet in enumerate(subnets):
        print("{:<6} {:<15} {:<15} {:<15}".format(
            i+1, str(subnet.network_address),
            str(subnet.broadcast_address),
            str(subnet.netmask)
        ))



 #Healthy practice in coding, for the near future when we work on big projects
if __name__ == "__main__":
    print("\nChoose a subnetting method:")
    print("1. Subnet based on required HOSTS per subnet")
    print("2. Subnet based on required NUMBER OF NETWORKS")

    choice = input("Enter 1 or 2: ").strip() #strip is a string method that removes extra spacing

    # Ask user based on the chosen method
    network, value = get_network_input(choice)

    if choice == '1':
        # value = hosts_per_subnet (keeps exact original logic)
        subnets, new_prefix = calculate_subnets(network, value)
        if subnets:
            print(f"\nOriginal Network: {network}")
            print(f"Required Prefix for {value} hosts: /{new_prefix}")
            print(f"Number of possible subnets: {len(subnets)}")
            display_subnets(subnets)

    elif choice == '2':
        # value = required number of networks
        subnets, new_prefix = calculate_networks(network, value)
        if subnets:
            print(f"\nOriginal Network: {network}")
            print(f"New Prefix to create {value} networks: /{new_prefix}")
            print(f"Number of created networks: {len(subnets)}")
            display_networks(subnets)

    else:
        print("Invalid choice. Please restart and enter 1 or 2.")



Choose a subnetting method:
1. Subnet based on required HOSTS per subnet
2. Subnet based on required NUMBER OF NETWORKS


Enter 1 or 2:  1
Enter network in CIDR format (e.g., 192.168.1.0/24):  195.10.30.0/24
Enter required number of hosts per subnet:  12



Original Network: 195.10.30.0/24
Required Prefix for 12 hosts: /28
Number of possible subnets: 16

Subnet     Network         Broadcast       Subnet Mask     Valid Hosts                  Usable Hosts
-----------------------------------------------------------------------------------------------
1      195.10.30.0     195.10.30.15    255.255.255.240 195.10.30.1 - 195.10.30.14 14             
2      195.10.30.16    195.10.30.31    255.255.255.240 195.10.30.17 - 195.10.30.30 14             
3      195.10.30.32    195.10.30.47    255.255.255.240 195.10.30.33 - 195.10.30.46 14             
4      195.10.30.48    195.10.30.63    255.255.255.240 195.10.30.49 - 195.10.30.62 14             
5      195.10.30.64    195.10.30.79    255.255.255.240 195.10.30.65 - 195.10.30.78 14             
6      195.10.30.80    195.10.30.95    255.255.255.240 195.10.30.81 - 195.10.30.94 14             
7      195.10.30.96    195.10.30.111   255.255.255.240 195.10.30.97 - 195.10.30.110 14             
8      195